
![AIF360](https://opengraph.githubassets.com/4b7ff9fccf4e6cafd9e59f42921b31a0f7f6696b009c8519e584be278094769d/Trusted-AI/AIF360)

In [1]:
# Instalar la librería AIF360

%pip install aif360
# pip install aif360["all"]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import de librerías

In [2]:

# elementales

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns

# sklearn

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, RocCurveDisplay, recall_score, precision_score, f1_score, roc_auc_score, roc_curve

# aif360

from aif360.sklearn.datasets import fetch_german
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.datasets import BinaryLabelDataset, StandardDataset



pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[inFairness]'


## Obtención y manipulación de los datos

Estaremos utilizando el dataset 'German Credit Data'.

In [3]:
# Obtenemos los datos.

X, y = fetch_german()
X.head()

checking_status  duration  \
sex    age   foreign_worker                             
male   aged  yes                         <0         6   
female young yes                   0<=X<200        48   
male   aged  yes                no checking        12   
             yes                         <0        42   
             yes                         <0        24   

                                             credit_history  \
sex    age   foreign_worker                                   
male   aged  yes             critical/other existing credit   
female young yes                              existing paid   
male   aged  yes             critical/other existing credit   
             yes                              existing paid   
             yes                         delayed previously   

                                         purpose  credit_amount  \
sex    age   foreign_worker                                       
male   aged  yes                        radio/tv           1169   
female young yes                        radio/tv           5951   
male   aged  yes                       education           2096   
             yes             furniture/equipment           7882   
             yes                         new car           4870   

                               savings_status employment  \
sex    age   foreign_worker                                
male   aged  yes             no known savings        >=7   
female young yes                         <100     1<=X<4   
male   aged  yes                         <100     4<=X<7   
             yes                         <100     4<=X<7   
             yes                         <100     1<=X<4   

                             installment_commitment other_parties  \
sex    age   foreign_worker                                         
male   aged  yes                                  4          none   
female young yes                                  2          none   
male   aged  yes                                  2          none   
             yes                                  2     guarantor   
             yes                                  3          none   

                             residence_since  ... age  other_payment_plans  \
sex    age   foreign_worker                   ...                            
male   aged  yes                           4  ...  67                 none   
female young yes                           2  ...  22                 none   
male   aged  yes                           3  ...  49                 none   
             yes                           4  ...  45                 none   
             yes                           4  ...  53                 none   

                              housing existing_credits                 job  \
sex    age   foreign_worker                                                  
male   aged  yes                  own                2             skilled   
female young yes                  own                1             skilled   
male   aged  yes                  own                1  unskilled resident   
             yes             for free                1             skilled   
             yes             for free                2             skilled   

                            num_dependents  own_telephone foreign_worker  \
sex    age   foreign_worker                                                
male   aged  yes                         1            yes            yes   
female young yes                         1           none            yes   
male   aged  yes                         2           none            yes   
             yes                         2           none            yes   
             yes                         2           none            yes   

                                sex marital_status  
sex    age   foreign_worker                         
male   aged  yes               male         single  
female young yes         

In [4]:
# Los manipularemos para dejarlos en el formato habitual.

df = (
    pd.concat([X, y], axis=1)
    .drop(columns=["sex", "foreign_worker"])
    .rename(columns={"age": "age_numeric"})
    .reset_index()
)
df.head()

,sex,age,foreign_worker,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,...,property_magnitude,age_numeric,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,marital_status,credit-risk
0,male,aged,yes,<0,6,critical/other existing credit,radio/tv,1169,no known savings,>=7,...,real estate,67,none,own,2,skilled,1,yes,single,good
1,female,young,yes,0<=X<200,48,existing paid,radio/tv,5951,<100,1<=X<4,...,real estate,22,none,own,1,skilled,1,none,div/dep/mar,bad
2,male,aged,yes,no checking,12,critical/other existing credit,education,2096,<100,4<=X<7,...,real estate,49,none,own,1,unskilled resident,2,none,single,good
3,male,aged,yes,<0,42,existing paid,furniture/equipment,7882,<100,4<=X<7,...,life insurance,45,none,for free,1,skilled,2,none,single,good
4,male,aged,yes,<0,24,delayed previously,new car,4870,<100,1<=X<4,...,no known property,53,none,for free,2,skilled,2,none,single,bad


In [5]:
df = df.rename(columns={"age": "age_cat", "age_numeric": "age"})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   sex                     1000 non-null   category
 1   age_cat                 1000 non-null   category
 2   foreign_worker          1000 non-null   category
 3   checking_status         1000 non-null   category
 4   duration                1000 non-null   int64   
 5   credit_history          1000 non-null   category
 6   purpose                 1000 non-null   category
 7   credit_amount           1000 non-null   int64   
 8   savings_status          1000 non-null   category
 9   employment              1000 non-null   category
 10  installment_commitment  1000 non-null   int64   
 11  other_parties           1000 non-null   category
 12  residence_since         1000 non-null   int64   
 13  property_magnitude      1000 non-null   category
 14  age                     1

### Descripción de las columnas.

**Atributos Numéricos**

- `duration`: Duración en meses. Rango (4,72).
- `credit_amount`: Cantidad de crédito solicitada. Rango (250, 18424) en DM - Marco alemán.
- `installment_commitment`: Tasa de cuota en porcentaje del ingreso disponible. Rango (1,4).
- `residence_since`: Tiempo de residencia actual. Rango (1,4).
- `age`: Edad en años. Rango (19, 75).
- `existing_credits`: Número de créditos existentes en este banco. Rango (1,4) en DM - Marco alemán.
- `num_dependents`: Número de personas responsables de proveer el mantenimiento. Rango (1,2).

**Atributos Categóricos**

- `checking_status`: Valores `'0<=X<200', '<0', '>=200', 'no checking'`.
- `credit_history`: Historial crediticio del solicitante. Valores `['all paid', 'critical/other existing credit', 'delayed previously', 'existing paid', 'no credits/all paid']`.
- `purpose`: Motivo por el cual el solicitante solicitó un préstamo. Valores `['business', 'domestic appliance', 'education', 'furniture/equipment', 'new car', 'other', 'radio/tv', 'repairs', 'retraining', 'used car']`.
- `savings_status`: Cuenta de ahorros/bonos. Valores `['100<=X<500', '500<=X<1000', '<100', '>=1000', 'no known savings']`.
- `employment`: Empleo actual desde (en años). Valores `['1<=X<4', '4<=X<7', '<1', '>=7', 'unemployed']`.
- `other_parties`: Otros deudores / garantes. Valores `['co applicant', 'guarantor', 'none']`.
- `property_magnitude`: Bienes del solicitante. Valores `['car', 'life insurance', 'no known property', 'real estate']`.
- `other_payment_plans`: Otros planes de pago a plazos. Valores `['bank', 'none', 'stores']`.
- `housing`: Situación de vivienda del solicitante. Valores `['for free', 'own', 'rent']`.
- `job`: Categorías de empleo definidas por el banco. Valores `['high qualif/self emp/mgmt', 'skilled', 'unemp/unskilled non res', 'unskilled resident']`.
- `own_telephone`: Si hay un teléfono registrado a nombre del cliente. Valores `['none', 'yes']`.
- `foreign_worker`: **Atributo protegido**. Valores `['no', 'yes']`.
- `sex`: **Atributo protegido**. Valores `['female', 'male']`.
- `marital_status`: Estado civil. Valores `['div/dep/mar', 'div/sep', 'mar/wid', 'single']`.

**Etiqueta**

- `credit-risk`: `'good'` (favorable) o `'bad'` (desfavorable).

## Pre-procesamiento

Haremos el pre-procesamiento de nuestros datos para poder trabajar con los algoritmos.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   sex                     1000 non-null   category
 1   age_cat                 1000 non-null   category
 2   foreign_worker          1000 non-null   category
 3   checking_status         1000 non-null   category
 4   duration                1000 non-null   int64   
 5   credit_history          1000 non-null   category
 6   purpose                 1000 non-null   category
 7   credit_amount           1000 non-null   int64   
 8   savings_status          1000 non-null   category
 9   employment              1000 non-null   category
 10  installment_commitment  1000 non-null   int64   
 11  other_parties           1000 non-null   category
 12  residence_since         1000 non-null   int64   
 13  property_magnitude      1000 non-null   category
 14  age                     1

In [7]:
# Binarización de algunas variables.

df["sex"] = df["sex"].map({"male": 1, "female": 0})
df["age_cat"] = df["age_cat"].map({"aged": 1, "young": 0})
df["foreign_worker"] = df["foreign_worker"].map({"no": 1, "yes": 0})
df["credit-risk"] = df["credit-risk"].map({"good": 1, "bad": 0})

In [8]:
# Separamos X e Y.

X = df.loc[:, df.columns != "credit-risk"]
y = df.loc[:, df.columns == "credit-risk"]

In [9]:
df1 = X.copy()

In [10]:
# Obtenemos variables dummies

ignore = ["sex", "age_cat", "foreign_worker"]
for catcol in df1.select_dtypes(include="category").columns:
    if catcol in ignore:
        pass
    else:
        dummies = pd.get_dummies(X[catcol])
        dummies.columns = [catcol + "_" + x for x in dummies.columns]
        X = pd.concat([X, dummies], axis=1).drop(columns=[catcol])

X.head()

,sex,age_cat,foreign_worker,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents,...,job_high qualif/self emp/mgmt,job_unemp/unskilled non res,job_unskilled resident,job_skilled,own_telephone_none,own_telephone_yes,marital_status_div/dep/mar,marital_status_div/sep,marital_status_mar/wid,marital_status_single
0,1,1,0,6,1169,4,4,67,2,1,...,False,False,False,True,False,True,False,False,False,True
1,0,0,0,48,5951,2,2,22,1,1,...,False,False,False,True,True,False,True,False,False,False
2,1,1,0,12,2096,2,3,49,1,2,...,False,False,True,False,True,False,False,False,False,True
3,1,1,0,42,7882,2,4,45,1,2,...,False,False,False,True,True,False,False,False,False,True
4,1,1,0,24,4870,3,4,53,2,2,...,False,False,False,True,True,False,False,False,False,True


In [11]:
# Datos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=908
)

In [12]:
# No utilizaremos la edad

X_train.drop(columns=["age"], inplace=True)
X_test.drop(columns=["age"], inplace=True)

## ¿Discriminación en nuestros datos?

Una forma de medir la discriminación en nuestros datos es a través de AIF360, que nos facilita el cálculo de métricas. Para ello debemos seguir una serie de pasos para instanciar un BinaryLabelDatasetMetric.

- #### **Statistical Parity Difference**.

Corresponde a la diferencia entre la probabilidad de pertenecer a la clase positiva dado que se pertenece a la clase no privilegiada y  la probabilidad de pertenecer a la clase positiva dado que se pertenece a la clase privilegiada, es decir:

$SP = Pr(Y = 1 | D = \text{unprivileged})- Pr(Y = 1 | D = \text{privileged})$

Valores negativos de esta métrica podrían indicar la presencia de sesgo.

- #### **Disparate Impact**.

Es una métrica para medir fairness que compara la proporción de individuos que reciben un resultado positivo para el grupo privilegiado y el grupo no privilegiado. El cálculo es la proporción del grupo no privilegiado que recibe un resultado positivo dividida por la proporción del grupo privilegiado que recibe un resultado positivo.


$DI = \frac{Pr(Y = 1 | D = \text{unprivileged})}{Pr(Y = 1 | D = \text{privileged})}$



In [13]:
# Necesitamos los datos de entrenamiento en conjunto con la etiqueta.

df_train = X_train.copy()
df_train["credit-score"] = y_train

### Métricas de Fairness para los datos: por categoría de edad.

In [14]:
# Debemos definir nuestros grupos privilegiados y no privilegiados.

privileged_groups = [{"age_cat": 1}]
unprivileged_groups = [{"age_cat": 0}]

In [15]:
# Instanciamos BinaryLabelDataset con edad como atributo protegido

attributes_params = dict(
    protected_attribute_names=["age_cat"], label_names=["credit-score"]
)
dt_train = BinaryLabelDataset(df=df_train, **attributes_params)

In [16]:
# Finalmente, la instancia de BinaryLabelDatasetMetric con la edad como atributo protegido.

metric_age_train = BinaryLabelDatasetMetric(
    dt_train,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

#### Métricas

In [17]:
print(
    f"Train Statistical Parity Difference (age): {metric_age_train.mean_difference()}"
)

Train Statistical Parity Difference (age): -0.13278706925580863


In [18]:
from aif360.explainers import MetricTextExplainer, MetricJSONExplainer
import pprint

In [19]:
MetricTextExplainer(metric_age_train).mean_difference()

'Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): -0.13278706925580863'

In [20]:
pprint.pp(MetricJSONExplainer(metric_age_train).disparate_impact())

('{"metric": "Disparate Impact", "message": "Disparate impact (probability of '
 'favorable outcome for unprivileged instances / probability of favorable '
 'outcome for privileged instances): 0.8178037886955184", '
 '"numPositivePredictionsUnprivileged": 90.0, "numUnprivileged": 151.0, '
 '"numPositivePredictionsPrivileged": 473.0, "numPrivileged": 649.0, '
 '"description": "Computed as the ratio of rate of favorable outcome for the '
 'unprivileged group to that of the privileged group.", "ideal": "The ideal '
 'value of this metric is 1.0 A value < 1 implies higher benefit for the '
 'privileged group and a value >1 implies a higher benefit for the '
 'unprivileged group."}')


**Pregunta**: Las métricas nos indican que existe sesgo con respecto a la edad en nuestros datos de entrenamiento. Supongamos que entrenamos un modelo con estos datos. ¿Que sucederá con estos sesgos dentro del modelo? ¿Serán replicados, ignorados o amplificados?

In [21]:
# Regresión logística

lr = LogisticRegressionCV(solver="liblinear", cv=10, random_state=908)
lr.fit(X_train, np.ravel(y_train))

LogisticRegressionCV(cv=10, random_state=908, solver='liblinear')

In [22]:
# Métricas modelo base

y_pred_proba = lr.predict_proba(X_test)[:, 1]
y_pred = y_pred_proba >= 0.5

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1: {f1_score(y_test, y_pred)}")

Accuracy: 0.74
Recall: 0.8686131386861314
F1: 0.8206896551724138


### Métricas de Fairness para las predicciones.




In [23]:
# Para facilitar el proceso


def get_aif_metrics(attr, df, label_names):

    privileged_groups = [{attr: 1}]
    unprivileged_groups = [{attr: 0}]

    attributes_params = dict(protected_attribute_names=[attr], label_names=label_names)
    dt = BinaryLabelDataset(
        df=df, **attributes_params, favorable_label=1, unfavorable_label=0
    )

    metric = BinaryLabelDatasetMetric(
        dt, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups
    )

    return metric, dt

#### Métricas de Fairness por categoría de edad.

In [24]:
metric_age_train, train_aifdf = get_aif_metrics("age_cat", df_train, ["credit-score"])
print(
    f"Train Statistical Parity Difference (age): {metric_age_train.mean_difference()}"
)
print(f"Train Disparate Impact (age): {metric_age_train.disparate_impact()}")

Train Statistical Parity Difference (age): -0.13278706925580863
Train Disparate Impact (age): 0.8178037886955184


In [25]:
y_train_pred_proba = lr.predict_proba(X_train)[:, 1]
y_train_pred = y_train_pred_proba >= 0.5

In [26]:
df_pred = X_train.copy()
df_pred["credit-score"] = y_train_pred

In [27]:
metric_age_pred, pred_aifdf = get_aif_metrics("age_cat", df_pred, ["credit-score"])
print(f"Pred Statistical Parity Difference (age): {metric_age_pred.mean_difference()}")
print(f"Pred Disparate Impact (age): {metric_age_pred.disparate_impact()}")

Pred Statistical Parity Difference (age): -0.18487943754528113
Pred Disparate Impact (age): 0.7710176431929628


In [28]:
orig_vs_pred = ClassificationMetric(
    train_aifdf,
    pred_aifdf,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

In [29]:
print(
    f"Error rate difference (unprivileged error rate - privileged error rate)= {orig_vs_pred.error_rate_difference()}\n"
)

print(
    f"False negative rate for privileged groups = {orig_vs_pred.false_negative_rate(privileged=True)}"
)
print(
    f"False negative rate for unprivileged groups = {orig_vs_pred.false_negative_rate(privileged=False)}"
)
print(f"False negative rate ratio = {orig_vs_pred.false_negative_rate_ratio()}\n")

print(
    f"False positive rate for privileged groups = {orig_vs_pred.false_positive_rate(privileged=True)}"
)
print(
    f"False positive rate for unprivileged groups = {orig_vs_pred.false_positive_rate(privileged=False)}"
)
print(f"False positive rate ratio = {orig_vs_pred.false_positive_rate_ratio()}")

Error rate difference (unprivileged error rate - privileged error rate)= 0.10803171460933281

False negative rate for privileged groups = 0.07188160676532769
False negative rate for unprivileged groups = 0.2222222222222222
False negative rate ratio = 3.0915032679738563

False positive rate for privileged groups = 0.48295454545454547
False positive rate for unprivileged groups = 0.39344262295081966
False positive rate ratio = 0.8146576663452266


## ¿Qué son los algoritmos de mitigación de sesgo?

Son un conjunto de técnicas que buscan ajustar las predicciones del modelo de acuerdo a varias definiciones estadísticas de Fairness, es decir, buscan disminuir el sesgo no deseado en los modelos.

Diferentes algoritmos para manejar el sesgo intervienen en distintas partes del ciclo de vida del modelo. Al respecto, tenemos 3 caminos para la meta de hacer predicciones justas:

- Fair Pre-Processing: Se basan en modificar los datos de entrenamiento del modelo. Entre ellos encontramos el algoritmo de Reweighing, que genera pesos para los las tuplas de los datos de entrenamiento dependiendo de su combinación de (grupo, etiqueta).

- Fair In-Processing: En este caso se modifica el algoritmo de aprendizaje. Uno de los más conocidos es Adversiarial Debiasing, donde el objetivo del predictor es maximizar el accuracy mientras se disminuye la capacidad del adversario para deducir el valor de un atributo protegido.

- Fair Post-Processing: En este caso se modifican las predicciones del algoritmo. Un ejemplo es el algoritmo de RejectOptionClassification, que se basa en la idea de que el sesgo es "más probable de ocurrir" cerca de los límites de la clasificación. Se entregan outcomes favorables al grupo no privilegiado y outcomes no favorables al grupo privilegiado dentro de un intervalo alrededor del límite de clasificación. Como consecuencia, obtendremos límites de clasificación distintos para cada grupo.

Entender cómo, cuando y por qué ocupar uno u otro algoritmo de manejo del sesgo es desafiante incluso para los expertos en el tema. Las preguntas que debemos responder al respecto son:

- ¿Deberíamos eliminar el sesgo de los datos de entrenamiento? (Pre-Processing)
- ¿Deberíamos crear nuevos clasificadores que aprenden modelos no sesgados? (In-Processing)
- ¿Es mejor corregir las predicciones del modelo? (Post-Processing)

El contexto dictará las ventajas y desventajas de ocupar uno u otro algoritmo. En este tutorial enseñamos la implementación del algoritmo EqOddsPostprocessing. Una de las ventajas de los algoritmos de Post-processing es que se pueden utilizar cuando no se tiene acceso directo al modelo en sí, sino solo a sus predicciones.

En el taller anterior ya entrenamos un modelo de Regresión Logística.


In [30]:
lr

LogisticRegressionCV(cv=10, random_state=908, solver='liblinear')

Obtengamos las predicciones para los datos de prueba, además de su accuracy:

In [31]:
y_pred_proba = lr.predict_proba(X_test)[:, 1]
y_pred = y_pred_proba >= 0.5

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.74


Ahora, añadimos las predicciones a los datos de testeo para formar un nuevo dataframe. Además restauraremos los datos de testeo originales (atributos + etiquetas).

In [32]:
# Dataframe test con predicciones
df_test_pred = X_test.copy()
df_test_pred["credit-score"] = y_pred

# Dataframe test con etiquetas
df_test = X_test.copy()
df_test["credit-score"] = y_test

Supongamos que en el contexto de nuestro problema nos interesa mejorar la igualdad de oportunidades (diferencia entre TPR) en nuestras predicciones. Veamos cuál es el valor de esta métrica en las predicciones originales del modelo:

In [33]:
metric_age_test_pred, test_pred_aifdf = get_aif_metrics(
    "age_cat", df_test_pred, ["credit-score"]
)
_, test_aifdf = get_aif_metrics("age_cat", df_test, ["credit-score"])

In [34]:
cm_pred_test = ClassificationMetric(
    test_aifdf,
    test_pred_aifdf,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

In [35]:
cm_pred_test.equal_opportunity_difference()

np.float64(-0.1974358974358975)

Un valor de 0 en `equal_opoortunity_difference` indica que no hay sesgo en la igualdad de oportunidades. Un valor negativo indica que el grupo privilegiado tiene una mayor oportunidad de ser clasificado correctamente que el grupo no privilegiado.

Ahora implementemos el algoritmo de EqOddsPostprocessing. Para esto, debemos instanciarlo de forma bastante similar a lo que hemos visto en las estructuras de AIF360, indicando grupos privilegiados y no privilegiados. Este algoritmo resuelve un problema de optimización lineal para cambiar las etiquetas en post de mejorar la igualdad en términos de TPR y FPR.

In [36]:
from aif360.algorithms.postprocessing.eq_odds_postprocessing import EqOddsPostprocessing

In [37]:
roc = EqOddsPostprocessing(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    seed=103,
)
roc.fit(test_aifdf, test_pred_aifdf)

In [38]:
df = roc.predict(test_pred_aifdf)

In [39]:
privileged_groups = [{"age_cat": 1}]
unprivileged_groups = [{"age_cat": 0}]


metric = BinaryLabelDatasetMetric(
    df, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups
)

In [40]:
cm_pred_test_post = ClassificationMetric(
    test_aifdf,
    df,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

In [41]:
cm_pred_test_post.equal_opportunity_difference()

np.float64(0.09786324786324785)

In [62]:
print(
    f"Error rate difference (unprivileged error rate - privileged error rate)= {cm_pred_test_post.error_rate_difference()}\n"
)

print(
    f"False negative rate for privileged groups = {cm_pred_test_post.false_negative_rate(privileged=True)}"
)
print(
    f"False negative rate for unprivileged groups = {cm_pred_test_post.false_negative_rate(privileged=False)}"
)
print(f"False negative rate ratio = {cm_pred_test_post.false_negative_rate_ratio()}\n")

print(
    f"False positive rate for privileged groups = {cm_pred_test_post.false_positive_rate(privileged=True)}"
)
print(
    f"False positive rate for unprivileged groups = {cm_pred_test_post.false_positive_rate(privileged=False)}"
)
print(f"False positive rate ratio = {cm_pred_test_post.false_positive_rate_ratio()}")

Error rate difference (unprivileged error rate - privileged error rate)= -0.06577480490523968

False negative rate for privileged groups = 0.24786324786324787
False negative rate for unprivileged groups = 0.15
False negative rate ratio = 0.6051724137931034

False positive rate for privileged groups = 0.6136363636363636
False positive rate for unprivileged groups = 0.42105263157894735
False positive rate ratio = 0.6861598440545809


In [42]:
lr

LogisticRegressionCV(cv=10, random_state=908, solver='liblinear')

In [43]:
df_train.head()

,sex,age_cat,foreign_worker,duration,credit_amount,installment_commitment,residence_since,existing_credits,num_dependents,checking_status_0<=X<200,...,job_unemp/unskilled non res,job_unskilled resident,job_skilled,own_telephone_none,own_telephone_yes,marital_status_div/dep/mar,marital_status_div/sep,marital_status_mar/wid,marital_status_single,credit-score
589,1,1,0,12,2246,3,3,2,1,False,...,False,False,True,True,False,False,False,False,True,0
336,0,0,0,13,2101,2,4,1,1,True,...,False,True,False,True,False,True,False,False,False,1
446,0,1,0,36,1842,4,4,1,1,False,...,False,False,True,False,True,True,False,False,False,0
997,1,1,0,12,804,4,4,1,1,False,...,False,False,True,True,False,False,False,False,True,1
917,1,1,0,6,14896,1,4,1,1,False,...,False,False,False,False,True,False,False,False,True,0


In [44]:
y_train_pred_proba = lr.predict_proba(X_train)[:, 1]
y_train_pred = y_train_pred_proba >= 0.5

In [45]:
df_pred = X_train.copy()
df_pred["credit-score"] = y_train_pred

In [46]:
metric_age_train, train_aifdf = get_aif_metrics("age_cat", df_train, ["credit-score"])
print(f"Statistical Parity Difference (age): {metric_age_train.mean_difference()}")
print(f"Disparate Impact (age): {metric_age_train.disparate_impact()}")

Statistical Parity Difference (age): -0.13278706925580863
Disparate Impact (age): 0.8178037886955184


In [47]:
metric_age_pred, pred_aifdf = get_aif_metrics("age_cat", df_pred, ["credit-score"])
print(f"Pred Statistical Parity Difference (age): {metric_age_pred.mean_difference()}")
print(f"Pred Disparate Impact (age): {metric_age_pred.disparate_impact()}")

Pred Statistical Parity Difference (age): -0.18487943754528113
Pred Disparate Impact (age): 0.7710176431929628


Los algoritmos de preprocesamiento tienden a cambiar las distribuciones de las variables protegidas en las muestras, o en términos más amplios, aplican cambios particulares a los datos para eliminar cualquier sesgo **en los datos de entrenamiento**. Estos cambios podrían ser, por ejemplo, cambiar la etiqueta de ciertos registros. El concepto fundamental es entrenar un modelo utilizando un conjunto de datos que ha sido "corregido".

En particular, estaremos aplicando la tecnica de **Reweighing**. Esta es una técnica de pre-procesamiento de datos que genera pesos para cada tupla según la combinación (grupo, etiqueta), para intentar mitigar el sesgo con respecto a los atributos sensibles antes de la clasificación.

En primer lugar, veamos como el Reweighing puede ayudar a eliminar el sesgo en los datos de entrenamiento.

In [48]:
from aif360.algorithms.preprocessing import Reweighing

In [49]:
# Originalmente todas las tuplas tienen peso 1.

train_aifdf.instance_weights[0:30]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

En este caso, estaremos enfocados en aplicar esta técnica para disminuir el sesgo con respecto a la categoría de edad.

In [50]:
# Debemos definir nuestros grupos privilegiados y no privilegiados.

privileged_groups = [{"age_cat": 1}]
unprivileged_groups = [{"age_cat": 0}]

# Instanciamos algoritmo de Reweighing

reweight = Reweighing(
    unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups
)

# Obtenemos los nuevos datos de entrenamiento (en formato BinaryLabelDataset)

reweighted = reweight.fit_transform(train_aifdf)

In [51]:
# Veamos que los pesos han cambiado

reweighted.instance_weights[0:30]

array([1.09242188, 1.18073611, 1.09242188, 0.96561047, 1.09242188,
       0.96561047, 1.09242188, 1.18073611, 0.73334016, 0.96561047,
       0.96561047, 0.96561047, 0.96561047, 0.96561047, 0.96561047,
       1.09242188, 0.96561047, 0.96561047, 1.09242188, 0.96561047,
       0.96561047, 0.96561047, 0.73334016, 0.96561047, 0.96561047,
       0.96561047, 1.09242188, 0.96561047, 0.96561047, 0.96561047])

In [52]:
type(reweight) == type(Reweighing(privileged_groups=[], unprivileged_groups=[]))

True

In [53]:
type(Reweighing)

aif360.decorating_metaclass.factory.<locals>.ApplyDecoratorMeta

In [54]:
# Miremos las métricas para estos datos

reweighted_metrics = BinaryLabelDatasetMetric(
    reweighted,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

In [55]:
print(
    f"Reweighted Train Statistical Parity Difference (age): {reweighted_metrics.mean_difference()}"
)

Reweighted Train Statistical Parity Difference (age): -3.3306690738754696e-16


In [56]:
# Regresión logística

lr = LogisticRegressionCV(solver="liblinear", cv=10, random_state=908)

# Al entrenar (fit) entregamos los pesos calculados por el algoritmo de Reweighing

lr.fit(X_train, np.ravel(y_train), sample_weight=reweighted.instance_weights)

LogisticRegressionCV(cv=10, random_state=908, solver='liblinear')

In [57]:
# ¿Cómo afectó la mitigación de sesgo al accuracy?

y_pred_proba = lr.predict_proba(X_test)[:, 1]
y_pred = y_pred_proba >= 0.5

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.715


In [58]:
df_test_rw = X_test.copy()
df_test_rw["credit-score"] = y_pred

In [59]:
metric_age_test_rw, rw_test_pred_aifdf = get_aif_metrics(
    "age_cat", df_test_rw, ["credit-score"]
)
print(
    f"Reweighted: Test Statistical Parity Difference (age): {metric_age_test_rw.mean_difference()}"
)
print(
    f"Reweighted: Test Disparate Impact (age): {metric_age_test_rw.disparate_impact()}"
)

Reweighted: Test Statistical Parity Difference (age): 0.02707437490046183
Reweighted: Test Disparate Impact (age): 1.032051282051282


In [60]:
cm_pred_test_rw = ClassificationMetric(
    test_aifdf,
    rw_test_pred_aifdf,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
)

In [61]:
cm_pred_test_rw.equal_opportunity_difference()

np.float64(0.04401709401709397)

[Documentación AIF360](https://aif360.readthedocs.io/en/stable/index.html)